# クラスタリング

## 凝集型クラスタリング

### 例題 3.1

In [16]:
import numpy as np
import math

D = [0, 1, 3, 5.5]

# 事例間の類似度は、差の逆数
def sim(d_k, d_l):
    diff = math.fabs(d_k - d_l)
    return 0 if diff <= 0.0 else (1 / diff)

def merge(lst, lhs_idx, rhs_idx):
    tmp = []

    for i in range(0, len(lst)):
        if (i is not lhs_idx and i is not rhs_idx):
            tmp = tmp + [lst[i]]
    
    return tmp + [lst[lhs_idx] + lst[rhs_idx]]

def aggolomerative_clustering(sim_fn):
    C = [[d] for d in D]
    
    print(C)

    while len(C) >= 2:
        indices = [(c_i, c_j) for c_i in enumerate(C)
                                           for c_j in enumerate(C)
                                           if c_j[1] is not c_i[1]]
        idx = np.argmax([sim_fn(c_i, c_j) for (_, c_i), (_, c_j) in indices])
        (c_m_idx, c_m), (c_n_idx, c_n) = indices[idx]
        C = merge(C, c_m_idx, c_n_idx)

        print(C)

# 単連結法による凝集型クラスタリング
def sim_single_link(c_i, c_j):
    return max([sim(x_k, x_l) for x_k in c_i for x_l in c_j])

print("aggolomerative clustering with single-link method:")
aggolomerative_clustering(sim_single_link)

# 完全連結法による凝集型クラスタリング
def sim_complete_link(c_i, c_j):
    return min([sim(x_k, x_l) for x_k in c_i for x_l in c_j])

print("aggolomerative clustering with complete-link method:")
aggolomerative_clustering(sim_complete_link)

aggolomerative clustering with single-link method:
[[0], [1], [3], [5.5]]
[[3], [5.5], [0, 1]]
[[5.5], [3, 0, 1]]
[[5.5, 3, 0, 1]]
aggolomerative clustering with complete-link method:
[[0], [1], [3], [5.5]]
[[3], [5.5], [0, 1]]
[[0, 1], [3, 5.5]]
[[0, 1, 3, 5.5]]


## k-平均法

### 例題 3.2

In [22]:
D = [0, 1, 3, 5.5]
k = 2

ms = [-1, 6]
old_ms = [math.nan, math.nan]

def convergence(ms, old_ms):
    return np.isclose(ms[0], old_ms[0])

while not convergence(ms, old_ms):
    clusters = [[], []]

    for x in D:
        c_max = np.argmax([sim(x, m) for m in ms])
        clusters[c_max] += [x]
    
    old_ms = ms
    ms = [np.mean(c) for c in clusters]
    
    print("clusters: %s" % clusters)
    print("new center: %s" % ms)

clusters: [[0, 1], [3, 5.5]]
new center: [0.5, 4.25]
clusters: [[0, 1], [3, 5.5]]
new center: [0.5, 4.25]


## EMアルゴリズム

1. Eステップ: 任意の$\mathbf{x}^{(i)} \in D$、任意の$c$について、$P(c|\mathbf{x}^{(i)}; \theta')$を計算する
2. Mステップ: $\theta^{max} = \underset{\theta}{\operatorname{argmax}}Q(\theta; \theta')$

ここでQ関数$Q(\theta, \theta')$は

$$
Q(\theta, \theta') = \sum_{\mathbf{x}^{(i)} \in D} \sum_c P(c|\mathbf{x}^{(i)}; \theta') \log P(c, \mathbf{x}^{(i)}; \theta)
$$

### 混合正規分布の場合

事例$\mathbf{x}$が各クラスタ$c$に正規分布にしたがって属すると仮定する、ただし分散$\sigma^2$は既知でありクラスタによって変わらないものとする。つまり$\mathbf{m}_c$をクラスタ$c$の平均ベクトルとすると、

$$
P(\mathbf{x}^{(i)}|c) = \frac{1}{\sqrt{(2 \pi \sigma^2)^d}} \exp(- \frac{|\mathbf{x}^{(i)} - \mathbf{m}_c|^2}{2 \sigma ^2})
$$

Eステップは

$$
\begin{align}
P(c|\mathbf{x}^{(i)}) &= \frac{P(\mathbf{x}^{(i)}, c)}{P(\mathbf{x}^{(i)})} = \frac{P(c, \mathbf{x}^{(i)})}{\sum_c P(c, \mathbf{x}{(i)})} = \frac{P(c)P(\mathbf{x}^{(i)}|c)}{\sum_c P(c)P(\mathbf{x}^{(i)}|c)} \\
&= \frac{P(c) \exp(- \frac{|\mathbf{x}^{(i)} - \mathbf{m}_c| ^2}{2 \sigma ^2})}{\sum_c P(c) \exp(- \frac{|\mathbf{x}^{(i)} - \mathbf{m}_c| ^2}{2 \sigma ^2})}
\end{align}
$$

Mステップは

$$
\begin{align}
Q(\theta, \theta') &= \sum_{\mathbf{x}^{(i)} \in D} \sum_c \overline{P}(c|\mathbf{x}^{(i)}; \theta') \log P(c, \mathbf{x}^{(i)}; \theta) \\
&= \sum_{\mathbf{x}^{(i)} \in D} \sum_c \overline{P}(c|\mathbf{x}^{(i)}; \theta') \log P(c) P(\mathbf{x}^{(i)}|c; \theta)  \\
&= \sum_{\mathbf{x}^{(i)} \in D} \sum_c \overline{P}(c|\mathbf{x}^{(i)}; \theta') \log P(c) \frac{1}{\sqrt{(2 \pi \sigma^2)^d}} \exp(- \frac{|\mathbf{x}^{(i)} - \mathbf{m}_c|^2}{2 \sigma ^2})
\end{align}
$$

これを最大化する$\mathbf{m}_c$を求めるために$\mathbf{m}_c$で偏微分すると

$$
\begin{align}
\frac{\partial Q(\theta, \theta')}{\partial \mathbf{m}_c} &= \frac{\partial}{\partial \mathbf{m}_c}\sum_{\mathbf{x}^{(i)} \in D} \sum_c \overline{P}(c|\mathbf{x}^{(i)}; \theta') \log P(c) \frac{1}{\sqrt{(2 \pi \sigma^2)^d}} \exp(- \frac{|\mathbf{x}^{(i)} - \mathbf{m}_c|^2}{2 \sigma ^2}) \\
&= \sum_{\mathbf{x}^{(i)} \in D} \overline{P}(c|\mathbf{x}^{(i)}; \theta') \left[ \frac{\partial}{\partial \mathbf{m}_c} \log P(c) + \frac{\partial}{\partial \mathbf{m}_c} \log \frac{1}{\sqrt{(2 \pi \sigma^2)^d}} + \frac{\partial}{\partial \mathbf{m}_c} \log \left\{\exp(- \frac{|\mathbf{x}^{(i)} - \mathbf{m}_c|^2}{2 \sigma ^2}) \right\} \right] \\
&= \sum_{\mathbf{x}^{(i)} \in D} \overline{P}(c|\mathbf{x}^{(i)}; \theta') \left[ \frac{\partial}{\partial \mathbf{m}_c} (- \frac{|\mathbf{x}^{(i)} - \mathbf{m}_c|^2}{2 \sigma ^2}) \right] \\
&= \sum_{\mathbf{x}^{(i)} \in D} \overline{P}(c|\mathbf{x}^{(i)}; \theta')  \frac{\mathbf{x}^{(i)} - \mathbf{m}_c}{\sigma ^2}
\end{align}
$$

これを0とおくと

$$
\begin{align}
\sum_{\mathbf{x}^{(i)} \in D} \overline{P}(c|\mathbf{x}^{(i)}; \theta')  \frac{\mathbf{x}^{(i)} - \mathbf{m}_c}{\sigma ^2} &= 0 \\
\sum_{\mathbf{x}^{(i)} \in D} \overline{P}(c|\mathbf{x}^{(i)}; \theta')  (\mathbf{x}^{(i)} - \mathbf{m}_c) &= 0 \\
\mathbf{m}_c = \frac{\sum_{\mathbf{x}^{(i)} \in D} \overline{P}(c|\mathbf{x}^{(i)}; \theta')\mathbf{x}^{(i)}}{\sum_{\mathbf{x}^{(i)} \in D} \overline{P}(c|\mathbf{x}^{(i)}; \theta')}
\end{align}
$$

EMアルゴリズムは、不完全データー非観測変数もしくは隠れ変数と呼ばれる、この場合ではクラスラベルcを持つデータに対し、尤度が大きくなるようにパラメータ、この場合は$\mathbf{m}_c$を推定する。